In [1]:
# Install dependencies
!pip install torch torchvision numpy pandas pillow



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np
import pandas as pd
import os


In [4]:
# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
model.eval()  # Set the model to evaluation mode


/Users/sksingh/knodax/knodax-latest-books/AWS-ML-Specialty/code-for-book/AWS-ML-Specialty-Code-For-Book/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sksingh/knodax/knodax-latest-books/AWS-ML-Specialty/code-for-book/AWS-ML-Specialty-Code-For-Book/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
# Define the image transformation pipeline
transform_pipeline = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet and VGG models typically expect 224x224
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [7]:
# Extract features from penultimate layer
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

def extract_features(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform_pipeline(image)
    image = image.unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        features = feature_extractor(image)

    # Flatten the features to a vector
    features = features.numpy().flatten()
    return features


In [8]:
# Path to your image folder
image_folder = 'pets-images'

# List all images
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Extract features and store in DataFrame
features_list = []
image_names = []

for img_name in image_files:
    img_path = os.path.join(image_folder, img_name)
    features = extract_features(img_path)
    features_list.append(features)
    image_names.append(img_name)

features_df = pd.DataFrame(features_list, index=image_names)


In [9]:
# Save as CSV
features_df.to_csv('image_features.csv')

# Save as NPY (optional)
np.save('image_features.npy', features_list)


In [15]:
# Verify saved CSV file
features_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
981.jpg,0.319490,0.352267,0.276600,0.294845,1.215281,0.286477,0.569022,0.165743,0.428349,0.360798,...,0.280150,0.358360,0.255263,1.008755,0.595700,0.246411,0.355889,0.127894,0.740858,0.153351
997.jpg,1.043161,0.222231,0.614841,0.230224,0.232043,0.386660,0.294602,0.823457,0.726833,0.336668,...,0.155789,0.550934,1.797481,1.314889,1.313294,0.264904,0.103622,0.100914,0.568956,0.132503
996.jpg,0.009009,0.327408,0.537036,0.216847,0.602143,0.303986,0.077241,1.431358,0.195319,0.468349,...,0.077158,0.423431,0.108331,0.084371,0.010501,0.319634,0.553130,0.290656,0.210739,0.034367
982.jpg,0.452215,0.269274,0.665673,0.613707,0.293034,0.184625,0.150129,0.530029,0.673436,0.255765,...,0.111165,0.618544,0.400971,0.177761,1.758728,0.259369,0.273899,0.230167,0.337444,0.071611
9963.jpg,0.446357,0.226816,0.683443,0.050764,0.445962,0.428702,0.736295,0.245775,0.357932,0.342968,...,0.908918,1.005140,1.133024,0.102726,0.334219,0.131507,1.057698,0.580108,0.311450,0.047659
